This updated Inference.ipynb notebook implements a robust pipeline for automated chest X-ray inference using a trained deep learning model. It loads and validates DICOM files, ensuring only chest X-rays with appropriate modality and body part are processed. Images are preprocessed (including resizing and normalization) to match the model’s input requirements, and predictions are made using the loaded model architecture and weights. The notebook outputs, interpretable results for each test DICOM, skipping and logging any files that do not meet validation criteria, and allows for easy adjustment of the decision threshold based on prior model evaluation. This workflow demonstrates a practical, reproducible approach for clinical AI inference and regulatory review.

In [6]:
 #Import necessary libraries
# Basic imports
import numpy as np
import pandas as pd
import os
import sys

# Deep learning imports
import tensorflow as tf
from tensorflow.keras.models import model_from_json

# Medical imaging imports
import pydicom

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline

# Suppress TensorFlow and warning messages
tf.get_logger().setLevel('ERROR')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Print versions for documentation
print(f"TensorFlow version: {tf.__version__}")
print(f"Python version: {sys.version.split()[0]}")
print(f"PyDICOM version: {pydicom.__version__}")

TensorFlow version: 2.1.0
Python version: 3.7.6
PyDICOM version: 1.4.2


In [7]:
import pydicom
import numpy as np
from tensorflow.keras.models import model_from_json

# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data

def check_dicom(filename): 
    try:
        ds = pydicom.dcmread(filename)
        # Check modality is X-ray (DX)
        if ds.Modality != 'DX':
            print(f"File {filename}: Not an X-ray (Modality={ds.Modality})")
            return None
            
        # Check body part is chest
        if hasattr(ds, 'BodyPartExamined') and ds.BodyPartExamined.lower() != 'chest':
            print(f"File {filename}: Not a chest X-ray (BodyPartExamined={ds.BodyPartExamined})")
            return None
            
        # Check image position (if available)
        if hasattr(ds, 'PatientPosition'):
            print(f"Image Position: {ds.PatientPosition}")
            if ds.PatientPosition not in ['PA', 'AP']:
                print(f"Warning: Unexpected patient position: {ds.PatientPosition}")
                
        # Additional DICOM info for validation
        print(f"Image Type: {ds.get('ImageType', 'Not specified')}")
        print(f"Patient Position: {ds.get('PatientPosition', 'Not specified')}")
            
        img = ds.pixel_array
        return img
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        return None

# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_mean, img_std, img_size): 
    # Resize image if needed
    from cv2 import resize, INTER_LINEAR
    if img.ndim == 2:
        img = np.stack([img]*3, axis=-1)  # Convert grayscale to 3-channel
    img_resized = resize(img, (img_size[2], img_size[1]), interpolation=INTER_LINEAR)
    img_resized = img_resized.astype(np.float32)
    # Normalize
    img_norm = (img_resized - img_mean) / img_std
    # Add batch dimension
    proc_img = np.expand_dims(img_norm, axis=0)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # Load model architecture
    with open(model_path, "r") as json_file:
        loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_json)
    # Load weights
    model.load_weights(weight_path)
    # Compile (use same loss/metrics as training)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    prob = model.predict(img)[0][0]
    prediction = int(prob > thresh)
    return prediction

In [8]:
# Add imports for better diagnostics
import os
import time
from tensorflow.keras.models import model_from_json

# Define model file paths first
model_path = "my_model_architecture.json"
weight_path = "xray_class_my_model.best.hdf5"

# Now check if files exist
print("Checking model files...")
print(f"Architecture file exists: {os.path.exists(model_path)}")
print(f"Weights file exists: {os.path.exists(weight_path)}")

# Verify files exist first
print("Checking model files...")
print(f"Architecture file exists: {os.path.exists(model_path)}")
print(f"Weights file exists: {os.path.exists(weight_path)}")

# Load model with timing and verbose output
try:
    print("\nLoading model architecture...")
    start_time = time.time()
    with open(model_path, "r") as json_file:
        loaded_model_json = json_file.read()
    print(f"Architecture loaded in {time.time() - start_time:.2f}s")
    
    print("\nCreating model from architecture...")
    start_time = time.time()
    my_model = model_from_json(loaded_model_json)
    print(f"Model created in {time.time() - start_time:.2f}s")
    
    print("\nLoading model weights...")
    start_time = time.time()
    my_model.load_weights(weight_path)
    print(f"Weights loaded in {time.time() - start_time:.2f}s")
    
    print("\nCompiling model...")
    my_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    print("Model compilation complete")
    
except Exception as e:
    print(f"Error during model loading: {str(e)}")
    raise

Checking model files...
Architecture file exists: True
Weights file exists: True
Checking model files...
Architecture file exists: True
Weights file exists: True

Loading model architecture...
Architecture loaded in 0.00s

Creating model from architecture...
Model created in 0.15s

Loading model weights...
Weights loaded in 0.11s

Compiling model...
Model compilation complete


In [9]:
# Make sure all functions are defined above this cell!

from skimage.transform import resize

def preprocess_image(img, img_mean, img_std, img_size): 
    # Convert grayscale to 3-channel if needed
    if img.ndim == 2:
        img = np.stack([img]*3, axis=-1)
    # Resize image to (height, width)
    img_resized = resize(img, (img_size[1], img_size[2]), preserve_range=True, anti_aliasing=True)
    img_resized = img_resized.astype(np.float32)
    # Normalize
    img_norm = (img_resized - img_mean) / img_std
    # Add batch dimension
    proc_img = np.expand_dims(img_norm, axis=0)
    return proc_img

test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model_architecture.json"
weight_path = "xray_class_my_model.best.hdf5"
IMG_SIZE = (1, 224, 224, 3)
img_mean = 128.0
img_std = 64.0
thresh = 0.1  # Use your chosen threshold

my_model = load_model(model_path, weight_path)

for i in test_dicoms:
    img = check_dicom(i)
    if img is None:
        print(f"Skipping {i}: not a valid chest X-ray.")
        continue
    img_proc = preprocess_image(img, img_mean, img_std, IMG_SIZE)
    pred = predict_image(my_model, img_proc, thresh)
    print(f"{i}: Prediction = {pred}")

Image Position: PA
Image Type: Not specified
Patient Position: PA
test1.dcm: Prediction = 0
Image Position: AP
Image Type: Not specified
Patient Position: AP
test2.dcm: Prediction = 1
Image Position: AP
Image Type: Not specified
Patient Position: AP
test3.dcm: Prediction = 1
File test4.dcm: Not a chest X-ray (BodyPartExamined=RIBCAGE)
Skipping test4.dcm: not a valid chest X-ray.
File test5.dcm: Not an X-ray (Modality=CT)
Skipping test5.dcm: not a valid chest X-ray.
Image Position: XX
Image Type: Not specified
Patient Position: XX
test6.dcm: Prediction = 0
